# GAN - FFHQ - 128×128 px
Generative Adversarial Network for generating images of faces from Flickr-Faces-HQ database - code for training GAN.

Developed by Daniel Konečný

## Initialize
Defines the basic libraries and initializes global variables needed in all codes. Connects the code to data source - Google Drive.

In [0]:
import numpy as np
import tensorflow as tf

dataset = 'ffhq'
x_dimension = 128
y_dimension = 128
note = ''

project_name = f'{dataset}{x_dimension}x{y_dimension}{note}'
project_path = '/'
dataset_path = f'{project_path}datasets/'
grid_path = f'{project_path}grids/'
raw_path = f'{project_path}raw/'
weight_path = f'{project_path}weights/'

latent_dimension = 256
batch_size = 256

## Load and preprocess training data
Loads all the images used for training and saves them as numpy nd-arrays.

**To prepare the dataset, it is necessary to have the original images saved in `raw` folder (in subfolders with size of 1000).**

### Compressed files (NPZ)

In [0]:
import matplotlib.image as mpimg

dataset_start = 0
dataset_size = 70000
dataset_step = 10000
raw_folder_size = 1000
dataset = np.empty((dataset_step, x_dimension, y_dimension, 3))

for dataset_index in range(dataset_start, dataset_size, dataset_step):
    for image_index in range(dataset_index, dataset_index + dataset_step):
        dataset[image_index - dataset_index] = mpimg.imread(
            f'{raw_path}'
            f'{(image_index//raw_folder_size)*raw_folder_size:05d}/'
            f'{image_index:05d}.png')
        if (image_index - dataset_index) % 100 == 0:
            print(f'Dataset index: {dataset_index}, Image index: {image_index}')
    np.savez_compressed(f'{dataset_path}{project_name}_dataset{dataset_index:05d}.npz', dataset)
    print(f'Dataset exported: {save_file}')

### Non-compressed files (NPY)

In [0]:
import matplotlib.image as mpimg

dataset_start = 0
dataset_size = 70000
dataset_step = 14000
raw_folder_size = 1000
dataset = np.empty((dataset_step, x_dimension, y_dimension, 3))

for dataset_index in range(dataset_start, dataset_size, dataset_step):
    for image_index in range(dataset_index, dataset_index + dataset_step):
        dataset[image_index - dataset_index] = mpimg.imread(
            f'{raw_path}'
            f'{(image_index//raw_folder_size)*raw_folder_size:05d}/'
            f'{image_index:05d}.png')
        if (image_index - dataset_index) % 100 == 0:
            print(f'Image {image_index:05d} processed.')
    np.save(f'{dataset_path}{project_name}_dataset{dataset_index:05d}.npy', dataset)
    print(f'Dataset {dataset_index:05d} exported.')

### Convert preloaded NPZ to NPY

In [0]:
start = 0
size = 70000
dataset_step = 1400
dataset = np.empty((dataset_step, x_dimension, y_dimension, 3))
loaded_dataset_step = 10000
loaded_dataset = np.empty((loaded_dataset_step, x_dimension, y_dimension, 3))

dataset_index = start
for loaded_dataset_index in range ((start // loaded_dataset_step) * loaded_dataset_step, size, loaded_dataset_step):
    loaded_samples = np.load(f'{dataset_path}{project_name}_dataset{loaded_dataset_index:05d}.npz')
    loaded_dataset = loaded_samples['arr_0']
    print(f'Dataset {loaded_dataset_index:05d} loaded.')
    while dataset_index < loaded_dataset_index + loaded_dataset_step:
        dataset[dataset_index % dataset_step] = loaded_dataset[dataset_index - loaded_dataset_index]
        dataset_index += 1
        if dataset_index % dataset_step == 0:
            np.save(f'{dataset_path}{project_name}_dataset'
                    f'{(dataset_index//dataset_step-1)*dataset_step:05d}.npy', dataset)
            print(f'Dataset {(dataset_index//dataset_step-1)*dataset_step:05d} exported.')

## Training functions
All functions necessary for training, initialize before.

In [0]:
import time
from tensorflow.keras import layers
from matplotlib import pyplot
from sklearn.datasets import fetch_lfw_people


dataset = None
dataset_size = 70000
dataset_step = 1400


def get_discriminator(image_shape=(x_dimension, y_dimension, 3)):
	discriminator = tf.keras.Sequential()
 
	discriminator.add(layers.Conv2D(32, (1, 1), padding='same', input_shape=image_shape))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	# None is the batch size.
	assert discriminator.output_shape == (None, x_dimension, y_dimension, 32)

	discriminator.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//2, y_dimension//2, 64)
 
	discriminator.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//4, y_dimension//4, 128)
 
	discriminator.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//8, y_dimension//8, 128)
 
	discriminator.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//16, y_dimension//16, 256)
 
	discriminator.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//32, y_dimension//32, 256)
 
	discriminator.add(layers.Flatten())
	discriminator.add(layers.Dense(1, activation='sigmoid'))
 
	discriminator_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	discriminator.compile(loss='binary_crossentropy',
	                      optimizer=discriminator_optimizer,
						  metrics=['accuracy'])
 
	return discriminator


def get_generator():
	generator = tf.keras.Sequential()
	
	generator.add(layers.Dense(x_dimension//32 * y_dimension//32 * 256, input_dim=latent_dimension))
	generator.add(layers.LeakyReLU(alpha=0.2))
	generator.add(layers.Reshape((x_dimension//32, y_dimension//32, 256)))
	assert generator.output_shape == (None, x_dimension//32, y_dimension//32, 256)

	generator.add(layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//16, y_dimension//16, 256)

	generator.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//8, y_dimension//8, 128)

	generator.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//4, y_dimension//4, 128)
 
	generator.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//2, y_dimension//2, 64)
 
	generator.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension, y_dimension, 32)
 
	generator.add(layers.Conv2D(3, (1, 1), activation='sigmoid', padding='same'))
	assert generator.output_shape == (None, x_dimension, y_dimension, 3)

	return generator


def get_gan(generator, discriminator):
	gan = tf.keras.Sequential()
 
	gan.add(generator)
	discriminator.trainable = False
	gan.add(discriminator)

	gan_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	gan.compile(loss='binary_crossentropy', optimizer=gan_optimizer)

	return gan


def get_latent_points(sample_count=1):
	latents = np.empty((sample_count, latent_dimension))

	for latents_index in range(sample_count):
		randoms = np.random.normal(0, 1, latent_dimension)
		normalizer = np.sum(randoms**2)**0.5
		latent = randoms/normalizer
		latents[latents_index] = latent
	
	return latents


def get_generated_images(generator, image_count):
	generated_latents = get_latent_points(image_count)
	generated_images = generator.predict(generated_latents)
	generated_labels = np.zeros((image_count, 1))
	return generated_images, generated_labels


def get_real_images(image_index, image_count):
	global dataset
	if image_index % dataset_step == 0:
		dataset_load_start_time = time.time()
		print(f'Loading {image_index:05d} dataset...')
		dataset = np.load(f'{dataset_path}{project_name}_dataset{image_index:05d}.npy')
		dataset_load_end_time = time.time()
		print(f'Dataset loaded in {dataset_load_end_time - dataset_load_start_time} s.')
	start_index = image_index%dataset_step
	end_index = (image_index+image_count)%dataset_step
	if end_index == 0:
		end_index = dataset_step
	real_images = dataset[start_index:end_index, :, :, :]
	np.random.shuffle(real_images)
	real_labels = np.ones((image_count, 1))
	return real_images, real_labels


def get_batch(image_index, generator):
	image_count = batch_size // 2
	if dataset_step - image_index % dataset_step < image_count:
		image_count = dataset_step - image_index % dataset_step

	real_images, real_labels = get_real_images(image_index, image_count)
	generated_images, generated_labels = get_generated_images(generator, batch_size-image_count)

	return np.vstack((real_images, generated_images)), np.vstack((real_labels, generated_labels))


def create_image_grid(examples, image_grid_size, epoch_index):
	for grid_index in range(image_grid_size * image_grid_size):
		pyplot.subplot(image_grid_size, image_grid_size, 1 + grid_index)
		pyplot.axis('off')
		pyplot.imshow(examples[grid_index])
	pyplot.savefig(f'{grid_path}{project_name}_grid{epoch_index+1:04d}.png')
	pyplot.show()


def evaluate_model(generator, discriminator, sample_count=100):
	random_index = np.random.randint(dataset_size-dataset_step+1, dataset_size-sample_count)
 
	real_images, real_labels = get_real_images(random_index, sample_count)
	_, real_accuracy = discriminator.evaluate(real_images, real_labels, verbose=0)
	
	generated_images, generated_labels = get_generated_images(generator, sample_count)
	_, generated_accuracy = discriminator.evaluate(generated_images, generated_labels, verbose=0)
	
	print(f'Accuracy real: {real_accuracy*100:.0f} %, generated: {generated_accuracy*100:.0f} %')


# train the generator and discriminator
def train(generator, discriminator, gan, first_epoch, epoch_count):
	image_grid_size = 5

	for epoch_index in range(first_epoch, epoch_count):
		epoch_start_time = time.time()
		image_index = 0
		while image_index < dataset_size:
			# Discriminator training.
			images, labels = get_batch(image_index, generator)
			discriminator_loss, _ = discriminator.train_on_batch(images, labels)
   
			# Generator training.
			latent_points = get_latent_points(batch_size)
			generated_labels = np.ones((batch_size, 1))
			generator_loss = gan.train_on_batch(latent_points, generated_labels)

			print(f'>{epoch_index+1}, {image_index}/{dataset_size}, d={discriminator_loss:.3f}, g={generator_loss:.3f}')

			image_index += batch_size // 2
			if image_index % dataset_step < batch_size // 2:
				print(f'Changing image index manually from {image_index} to {image_index // dataset_step * dataset_step}')
				image_index = image_index // dataset_step * dataset_step
   
		evaluate_model(generator, discriminator)
		generator.save_weights(f'{weight_path}{project_name}_generator{epoch_index+1:04d}.h5')
		discriminator.save_weights(f'{weight_path}{project_name}_discriminator{epoch_index+1:04d}.h5')
		gan.save_weights(f'{weight_path}{project_name}_gan{epoch_index+1:04d}.h5')
  
		if (epoch_index + 1) % 10 == 0:
			display_latent = get_latent_points(image_grid_size*image_grid_size)
			display_images = generator.predict(display_latent)
			create_image_grid(display_images, image_grid_size, epoch_index)
		
		epoch_end_time = time.time()
		print(f'Epoch {epoch_index+1} finished in {epoch_end_time - epoch_start_time} s.')

## Training
Set epoch count and launch for training of the GAN.

In [0]:
epoch_count = 1000
first_epoch = 1000

print("Creating GAN...")
discriminator = get_discriminator()
generator = get_generator()
gan = get_gan(generator, discriminator)
if first_epoch > 0:
    discriminator.load_weights(f'{weight_path}{project_name}_discriminator{first_epoch:04d}.h5')
    generator.load_weights(f'{weight_path}{project_name}_generator{first_epoch:04d}.h5')
    gan.load_weights(f'{weight_path}{project_name}_gan{first_epoch:04d}.h5')

print("Training...")
train(generator, discriminator, gan, first_epoch, epoch_count)

## Summary
Launch to display the summary of Discriminator and Generator models.

In [0]:
print('\x1b[0;32;40m' + 'DISCRIMINATOR' + '\x1b[0m')
discriminator = get_discriminator()
print(f'{discriminator.summary()}\n')

print('\x1b[0;32;40m' + 'GENERATOR' + '\x1b[0m')
generator = get_generator()
print(f'{generator.summary()}\n')